In [1]:
%pwd

'd:\\Documentation_sample\\research'

In [2]:
import os

os.chdir("D:\Documentation_sample\Documentation_sample")

FileNotFoundError: [WinError 2] The system cannot find the file specified: 'D:\\Documentation_sample\\Documentation_sample'

In [ ]:
%pwd

'D:\\Documentation_sample\\Documentation_sample'

In [ ]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_batch_size : int
    params_valid_size : float
    paranms_Loss_function : str
    params_learning_rate : float
    params_momentum: float
    params_image_dim : int

In [4]:
from sample_project.constants import *
from sample_project.utils.common import read_yaml, create_directories

ModuleNotFoundError: No module named 'sample_project'